In [1]:
import requests

In [2]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [3]:
data = requests.get(standings_url)

In [4]:
from bs4 import BeautifulSoup

In [5]:
soup = BeautifulSoup(data.text)

In [6]:
standings_table=soup.select('table.stats_table')[0] #select the table

IndexError: list index out of range

In [ ]:
links = standings_table.find_all('a') #find all the links for each squad

In [ ]:
links = [l.get("href") for l in links]

In [ ]:
links = [l for l in links if '/squads/' in l]

In [ ]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [ ]:
team_url = team_urls[0]

In [ ]:
data = requests.get(team_url)

In [ ]:
import pandas as pd

matches = pd.read_html(data.text, match="Scores & Fixtures")

In [ ]:
soup = BeautifulSoup(data.text)

In [ ]:
links = soup.find_all('a')

In [ ]:
links = [l.get("href") for l in links]

In [ ]:
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [ ]:
data = requests.get(f"https://fbref.com{links[0]}")

In [ ]:
shooting = pd.read_html(data.text, match="Shooting")[0]

In [ ]:
shooting.columns=shooting.columns.droplevel()

In [ ]:
team_data = matches[0].merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on ="Date")

In [ ]:
years = list(range(2023,2020, -1))

In [ ]:
all_matches = []

In [ ]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [ ]:
import time
for year in years:
    data = requests.get(standings_url) #get the standings_url
    soup = BeautifulSoup(data.text)#parse
    standings_table = soup.select('table.stats_table')[0] #get the table holding each team info

    links = [l.get("href") for l in standings_table.find_all('a')] #get the href property for links
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links] #turn into absolute links
    
    previous_season = soup.select("a.prev")[0].get("href") #gets anchor tags with prev
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ") #set team names properly
        
        data = requests.get(team_url) #get the team url
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0] #read into matches
        
        soup = BeautifulSoup(data.text) #parse
        links = [l.get("href") for l in soup.find_all('a')] #pull out all of the links
        links = [l for l in links if l and 'all_comps/shooting/' in l] #gets the shooting links
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0] #read in shooting stats
        shooting.columns = shooting.columns.droplevel() #drop the top index level
        
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date") #merge shooting stats with the team data
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"] #filter out non-league competitions
        
        team_data["Season"] = year # add in a season column
        team_data["Team"] = team_name #add in a team column
        all_matches.append(team_data)
        time.sleep(1)
        
        


In [ ]:
len(all_matches)

In [ ]:
match_df = pd.concat(all_matches)

In [ ]:
match_df.columns = [c.lower() for c in match_df.columns]

In [ ]:
match_df

In [ ]:
match_df.to_csv("matches.csv")